In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/Shared\ drives/ecl-pedro/video-coding-utils/notebooks/

---

In [3]:
from utils.utils import *
from utils import models

In [4]:
import tensorflow as tf

In [5]:
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.image import psnr
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [8]:
def PSNR(x,y):
    return psnr(x,y,max_val=1)

In [56]:
ds = tf.data.Dataset.from_tensor_slices(list(range(50000)))

In [57]:
ds = ds.shuffle(50000)

In [58]:
def parse_function(xdir, ydir):
    
    def fun(i):
        
        i = tf.strings.as_string(i)
        fcompressed = tf.io.read_file(xdir + i + '.png')
        compressed = tf.image.decode_png(fcompressed, channels=1)
        compressed = tf.image.convert_image_dtype(compressed, tf.float32)
        compressed = tf.image.resize(compressed, [64, 64])
        
        fgroundtruth = tf.io.read_file(ydir + i + '.png')
        groundtruth = tf.image.decode_png(fgroundtruth, channels=1)
        groundtruth = tf.image.convert_image_dtype(groundtruth, tf.float32)
        groundtruth = tf.image.resize(groundtruth, [64, 64])
        
        return compressed, groundtruth
    
    return fun

In [59]:
ds = ds.map(parse_function('../../data/preprocessed/basketball/compressed/', '../../data/preprocessed/basketball/ground_truth/'))

In [60]:
bds = ds.batch(128)

In [61]:
model = models.ARCNN()

In [62]:
model = Sequential([
    Conv2D(64, 9, activation='relu', padding='same', input_shape=(64,64,1), name='conv1'), # 64 9x9 filters, ReLU activation
    Conv2D(32, 7, activation='relu', padding='same', name='conv2'),                        # 32 7x7 filters, ReLU activation
    Conv2D(16, 1, activation='relu', padding='same', name='conv3'),                        # 16 1x1 filters, ReLU activation
    Conv2D(1, 5, activation=None, padding='same', name='conv4'),                           # 1  5x5 filter,    no activation
], name='AR-CNN')

# originally "replicate" padding

In [63]:
model.summary()

Model: "AR-CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 64, 64, 64)        5248      
_________________________________________________________________
conv2 (Conv2D)               (None, 64, 64, 32)        100384    
_________________________________________________________________
conv3 (Conv2D)               (None, 64, 64, 16)        528       
_________________________________________________________________
conv4 (Conv2D)               (None, 64, 64, 1)         401       
Total params: 106,561
Trainable params: 106,561
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.compile(
    optimizer='SGD',
    loss=MSE,
    metrics=[PSNR]
)

In [65]:
ckpt_callback = ModelCheckpoint(
    filepath='checkpoints/ARCNN_tfdata',
    save_weights_only=True,
    verbose=0
)

In [66]:
history = model.fit(
    bds,
    callbacks=ckpt_callback
)

  3/391 [..............................] - ETA: 18:13 - loss: 0.1895 - PSNR: 7.4079

KeyboardInterrupt: 